In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import xgboost as xgb
import numpy as np
import shap
from tqdm import tqdm
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import roc_curve, auc, f1_score, roc_auc_score, accuracy_score
import lightgbm as lgbm
from imblearn.over_sampling import SMOTE, SVMSMOTE
import random

PARK_DATA_PATH = './exports'
%matplotlib inline


In [2]:
SEED = 42398

np.random.seed(SEED)
random.seed(SEED)

In [3]:
survey = pd.read_csv('survey_latest.csv')
survey = survey.rename(columns={"id": "fileID"})
survey.head()

fileID                   email  gender   age  \
0  2020-06-27T16-09-18-371Z17  srishtiraj12@gmail.com  female  25.0   
1  2020-06-25T05-37-40-785Z52                     NaN    male  89.0   
2   2020-06-16T16-38-56-595Z6  jaspreet1597@gmail.com  female  22.0   
3  2020-05-31T21-23-46-040Z66                     NaN  female  67.0   
4  2020-05-28T23-18-14-039Z54        kc2vrd@gmail.com    male  59.0   

         country english-first-language previous-video-calling  \
0          India                    yes                    yes   
1      Australia                    yes                    yes   
2  United States                     no                    yes   
3  United States                    yes                    yes   
4  United States                    yes                    yes   

  previous-automated-analysis diagnosed year-of-diagnosis  ...  \
0                          no        no               NaN  ...   
1                          no       yes              2019  ...   
2                          no        no               NaN  ...   
3                          no       yes              2005  ...   
4                          no       yes              2015  ...   

  understood-instructions  use-in-future    no-problems videos-helpful  \
0           stronglyAgree  stronglyAgree  stronglyAgree  stronglyAgree   
1                   agree          agree       disagree          agree   
2           stronglyAgree  stronglyAgree  stronglyAgree  stronglyAgree   
3           stronglyAgree  stronglyAgree          agree  stronglyAgree   
4           stronglyAgree          agree  stronglyAgree  stronglyAgree   

  prior-task-knowledge likes dislikes  \
0             disagree   NaN      NaN   
1                agree   NaN      NaN   
2             disagree   NaN      NaN   
3        stronglyAgree   NaN      NaN   
4        stronglyAgree   NaN      NaN   

                                           strengths improvements repeat  
0  its easy and efficient layout was quite helpfu...          NaN     no  
1                  I would like to know how I rated.          NaN     no  
2  I like the website design, it was very easy an...          NaN     no  
3  The tasks were very clear, you were given time...          NaN     no  
4      didn't have to leave the house to participate          NaN     no  

[5 rows x 31 columns]

In [4]:
data = pd.read_csv("./exports/MAX-PRAAT.csv")
data.head()

fileID       f0m       f0j      f0jr      ashr  \
0  2017-08-18T14-59-52-530Z49 -0.015835  0.466943  0.875847  1.225285   
1  2017-08-18T15-24-14-004Z53 -0.731817 -0.629661 -0.501102  0.698558   
2  2017-08-22T02-01-21-948Z87  2.020570 -0.048985  1.313108 -0.302426   
3  2017-09-22T18-38-44-872Z33  1.108304 -0.776772  0.088942 -0.851917   
4  2017-09-28T14-17-07-280Z18  0.855183  0.766868  0.767567 -0.669065   

      cepm0     cepm1     cepm2     cepm3     cepm4  ...  relbandpower3  \
0  0.170057 -0.288160 -0.792688 -0.279520  0.965751  ...       0.545367   
1  0.492770  0.716622 -0.495151  1.343956 -0.688948  ...       0.490960   
2  0.229702 -0.204992 -0.949751  0.894819  0.036164  ...       1.178542   
3 -0.409584 -0.351290  0.448398 -0.846547  0.795356  ...      -0.151484   
4 -0.254265 -0.167467  0.442004 -0.467342  0.489033  ...       0.141626   

      f0std  meanF0Hz  stdevF0Hz       HNR  localabsoluteJitter  ddpJitter  \
0  0.287669  0.129776  -0.478062  0.321177            -0.552437  -0.460989   
1 -0.466770 -0.114590   1.158259 -0.284127            -0.142941  -0.222074   
2 -0.369418  1.348527  -0.595546 -0.167789            -0.253662   0.675451   
3 -0.358133  1.286718  -1.066627  0.280646            -1.038789  -0.610898   
4  0.461507  1.112552  -0.044869 -0.764459             0.129300   1.355003   

   apq11Shimmer  ddaShimmer  PD  
0     -0.008952   -0.157510   1  
1      0.731128    0.965254   0  
2     -0.051229    0.428214   1  
3     -0.178564   -0.226887   1  
4      0.116886    1.113822   1  

[5 rows x 47 columns]

In [5]:
INDEX = "fileID"
LABEL = "PD"
DATA_COLUMNS = data.columns.to_list()
DATA_COLUMNS = list(filter(lambda column: column not in [INDEX, LABEL], DATA_COLUMNS))

In [6]:
data = pd.merge(data, survey, on=INDEX, how="inner")
data.head()

fileID       f0m       f0j      f0jr      ashr  \
0  2017-08-18T14-59-52-530Z49 -0.015835  0.466943  0.875847  1.225285   
1  2017-08-18T15-24-14-004Z53 -0.731817 -0.629661 -0.501102  0.698558   
2  2017-08-22T02-01-21-948Z87  2.020570 -0.048985  1.313108 -0.302426   
3  2017-09-22T18-38-44-872Z33  1.108304 -0.776772  0.088942 -0.851917   
4  2017-09-28T14-17-07-280Z18  0.855183  0.766868  0.767567 -0.669065   

      cepm0     cepm1     cepm2     cepm3     cepm4  ...  \
0  0.170057 -0.288160 -0.792688 -0.279520  0.965751  ...   
1  0.492770  0.716622 -0.495151  1.343956 -0.688948  ...   
2  0.229702 -0.204992 -0.949751  0.894819  0.036164  ...   
3 -0.409584 -0.351290  0.448398 -0.846547  0.795356  ...   
4 -0.254265 -0.167467  0.442004 -0.467342  0.489033  ...   

   understood-instructions  use-in-future  no-problems  videos-helpful  \
0            stronglyAgree  stronglyAgree      neither   stronglyAgree   
1                  neither          agree      neither           agree   
2            stronglyAgree          agree        agree           agree   
3                    agree  stronglyAgree        agree   stronglyAgree   
4            stronglyAgree  stronglyAgree        agree   stronglyAgree   

   prior-task-knowledge                                              likes  \
0         stronglyAgree                                                NaN   
1                 agree                                                NaN   
2                 agree  It's fun to watch myself though I didn't get m...   
3                 agree                         easy to use AND UNDERSTAND   
4                 agree                    easy to use, immediate feedback   

                                        dislikes  \
0                                            NaN   
1                                            NaN   
2  Figuring out the correct angles for my hands.   
3                                        nothing   
4                                    using a mac   

                                           strengths  \
0                                                NaN   
1                                                NaN   
2  The tasks were easy to do, and I wonder how ch...   
3  could do at my own pace at home and without ha...   
4                                                NaN   

                                        improvements  repeat  
0                                                NaN     NaN  
1                                                NaN     NaN  
2  I'm not sure of suggestions yet. Maybe some fe...     NaN  
3  how to understand and use the computer technol...     NaN  
4  screen shots should include stop recording, no...     NaN  

[5 rows x 77 columns]

In [7]:
data = data[data["repeat"] != "yes"]

lab = data[data["fileID"].str.contains("NIH")]
wild = data[~data["fileID"].str.contains("NIH")]

In [8]:
targets = data[LABEL]

data = data[DATA_COLUMNS]
lab = lab[DATA_COLUMNS]
wild = wild[DATA_COLUMNS]

In [9]:
from scipy import stats

for column in DATA_COLUMNS:
    # Apply t-test on every column
    result = stats.ttest_ind(lab[column], wild[column])
    print("Variable {}: {}".format(column, result))

Variable f0m: Ttest_indResult(statistic=2.410343610341521, pvalue=0.016185786344197733)
Variable f0j: Ttest_indResult(statistic=5.039219680664693, pvalue=5.9100555430631e-07)
Variable f0jr: Ttest_indResult(statistic=5.842865348910786, pvalue=7.754645983991762e-09)
Variable ashr: Ttest_indResult(statistic=-4.3430971635186095, pvalue=1.6053384408287977e-05)
Variable cepm0: Ttest_indResult(statistic=-6.9409040500470685, pvalue=8.666143185528109e-12)
Variable cepm1: Ttest_indResult(statistic=5.527547269627043, pvalue=4.534915744454889e-08)
Variable cepm2: Ttest_indResult(statistic=3.6420355436298597, pvalue=0.0002897812024245347)
Variable cepm3: Ttest_indResult(statistic=-11.211508051808991, pvalue=5.189224409776068e-27)
Variable cepm4: Ttest_indResult(statistic=0.5784759566388351, pvalue=0.5631227225145903)
Variable cepm5: Ttest_indResult(statistic=0.37982885950163076, pvalue=0.7041838480837249)
Variable cepm6: Ttest_indResult(statistic=-0.6873913378422629, pvalue=0.49205628868683515)
Var

In [10]:
top_feature_list = ['cepj5', 'cepm8', 'Hnorm', 'cepm4', 'cepj0', 'relbandpower1', 'cepm5', 'ppe', 'cepj3', 'cepm7', 'ashr', 'HNR']

In [11]:
for feature_idx in range(1, len(top_feature_list)):
    feature_list = top_feature_list[:feature_idx]

    X = data[feature_list].to_numpy()
    y = targets.to_numpy()

    labels = []
    predictions = []
    probs = []

    loo = LeaveOneOut()

    for train_index, test_index in tqdm(loo.split(X), total=len(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        smote = SVMSMOTE()
        X_train, y_train = smote.fit_resample(X_train, y_train)

        model = xgb.XGBClassifier()
        model.fit(X_train, y_train)

        label = y_test
        prediction = model.predict(X_test)
        prob = model.predict_proba(X_test)

        labels.append(label)
        predictions.append(prediction)
        probs.append(prob)
    
    predictions = np.squeeze(predictions)
    probs = np.squeeze(probs)
    labels = np.squeeze(labels)

    print("ROC AUC: {}".format(roc_auc_score(labels, probs[:, 1])))
    print("Accuracy: {}".format(accuracy_score(labels, predictions)))


  0%|          | 2/726 [00:00<00:47, 15.24it/s]ROC AUC: 0.6249341932087391
Accuracy: 0.5798898071625345
  0%|          | 2/726 [00:00<00:46, 15.70it/s]ROC AUC: 0.6609963148196895
Accuracy: 0.6322314049586777
  0%|          | 2/726 [00:00<00:50, 14.23it/s]ROC AUC: 0.6638095551460912
Accuracy: 0.6460055096418733
  0%|          | 2/726 [00:00<00:58, 12.31it/s]ROC AUC: 0.7104254409055015
Accuracy: 0.6721763085399449
  0%|          | 2/726 [00:00<00:51, 14.06it/s]ROC AUC: 0.7222788891813635
Accuracy: 0.6914600550964187
 17%|█▋        | 120/726 [00:09<00:48, 12.40it/s]


KeyboardInterrupt: 